Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as any collaborators you worked with:

In [ ]:
COLLABORATORS = "Zala Bhan, Anjali Prasad, Eva Cullen"

## To receive credit for this assignment, you must also fill out the [AI Use survey](https://forms.gle/ZhR5k8TdAeN8rj4CA)


---

In [ ]:
%matplotlib inline
%precision 16
import numpy
import matplotlib.pyplot as plt
import pandas as pd

# Final Project

This notebook will provide a brief structure and rubric for presenting your final project. 

The purpose of the project is 2-fold
* To give you an opportunity to work on a problem you are truly interested in (as this is the best way to actually learn something)
* To demonstrate to me that you understand the overall workflow of problem solving from problem selection to implementation to discussion 

You can choose any subject area that interests you as long as there is a computational component to it.  However, please do not reuse projects or homeworks you have done in other classes.  This should be **your** original work.

**You can work in teams, but clearly identify each persons contribution** and every team member should hand in their own copy of the notebook.

### Structure
There are 5 parts for a total of 100 points that provide the overall structure of a mini research project.

* Abstract
* Introduction and Problem Description
* Brief discussion of Computational approach and import of any additional packages
* Implementation including tests
* Discussion of results and future directions

For grading purposes, please try to make this notebook entirely self contained. 

The project is worth about 2 problem sets and should be of comparable length (please: I will have about 100 of these to read and I am not expecting full 10 page papers).  The actual project does not necessarily have to work but in that case you should demonstrate that you understand why it did not work and what steps you would take next to fix it.

Have fun

## Abstract [10 pts]

Provide a 1-2 paragraph abstract of the project in the style of a research paper.  The abstract should contain

* A brief description of the problem
* A brief justification describing why this problem is important/interesting to you
* A general description of the computational approach
* A brief summary of what you did and what you learned


Cardiovascular diseases remain a leading cause of morbidity and mortality worldwide, and understanding blook flow behavior in compliant, viscoelastic vessels is critical for diagnosis, treatment planning, and medical device design. Consequently, developing efficient and accurate computational methods can provide clinicians and researchers with valuable insights into disease progression, surgical outcomes, and personalized medicine. The frequency-domain spectral element method (SEM) offers a computational advantage over the traditional finite element method (FEM) while maintaining high accuracy for the purposes of application.

## Introduction [15 pts]

In ~4-5 paragraphs, describe 
* The general problem you want to solve
* Why it is important and what you hope to achieve.

Please provide basic **references**, particularly if you are reproducing results from a paper. Also include any basic equations you plan to solve. 

Please use proper spelling and grammar. 

### References

<sup>1</sup><span id="Lee and Jang SEM"> https://www.sciencedirect.com/science/article/abs/pii/S0096300312000264?via%3Dihub</span>

<sup>2</sup><span id="Bessems et al."> https://www.sciencedirect.com/science/article/abs/pii/S0021929007003922?via%3Dihub</span>

## Computational  Methods [10 pts]

Describe the specific approach you will take to solve some concrete aspect of the general problem. 

You should  include all the numerical or computational methods you intend to use.  These can include methods or packages  we did not discuss in class but provide some reference to the method. You do not need to explain in detail how the methods work, but you should describe their basic functionality and justify your choices. 




The following derivation for the frequency-domain SEM is taken from Lee and Jang (2012).<sup>1</sup> Implementing the SEM involves initial mathematical formulation and demonstration and validation cases. To simplify model formulation, we:
1. Drop vicoelasticity. i.e. we model the vessel wall as purely elastic, this would require us to re-derive the constitutive equation in the purely elastic limit (by setting relaxation times to zero).
2. Start by dropping nonlinear terms. i.e. we solve the linear governing equations, eliminating the need for an alternating frequency-time (AFT) method-based direct iteration method. We include the nonlinear terms if time allows?
3. Use a single, uniform element. i.e. we model the entire vessel as one spectral element and do not apply the "global system equation" utilized in the paper.
4. Do not verify the model using real, complex datasets.

We begin by defining the time-domain governing equations, comprising the continuity equation, the momentum balance equation, and the constitutive equation. The basis for this mathematical model is derived form from Bessem et al.<sup>2</sup> and assumes that blood is a Newtonian fluid that can be represented by three governing equations.<sup>1</sup> The continuity equation and momentum balance equation are given by 

$$
\frac{\delta A}{\delta t} + \frac{\delta Q}{\delta t} = -\psi \tag{continuity equation}
$$

and

$$
\frac{\delta Q}{\delta t} + \frac{\delta(\gamma Q^2 A^{-1})}{\delta x} + \rho^{-1} A \frac{\delta P}{\delta x}
= Af_x - \nu \eta Q A^{-1} + \nu \frac{\delta^2 Q}{\delta x^2} \tag{momentum balance equation}
$$

in which $Q(x,t)$ is the volumetric blood flow rate, $P(x,t)$ is the blood pressure, $A(x,t)$ is the lumen cross-sectional area of the vessel, $\psi$ is the outflow function which represents the outflow across the permeable vessel wall, $f_x$ is the body force acting on the blood fluid in the $x$-direction, $\rho$ is the mass density of blood fluid, and $\nu$ is the kinematic viscosity of blood fluid.<sup>1</sup> $\gamma$ and $\eta$ are parameters determined by the flow velocity profile across the cross-section of a vessel, and are given by 
$$
\begin{aligned}
\gamma &= 0 & \eta &= 0 && \text{(uniform flow)} \\
\gamma &= \frac{1}{3} & \eta &= 8\pi && \text{(parabolic flow)} \\
\gamma &= \frac{2}{3}R^{-1} \delta & \eta &= 2\pi R \delta^{-1} && \text{(boundary-layered flow)}
\end{aligned}
$$

The constitutive equation is given by
$$
Rh^{-1}(P-P_d) + \tau_{\epsilon} \frac{\delta(Rh^{-1}(P-P_d))}{\delta t}
= \frac{E}{2}(1-\mu^2)^{-1} \left[(A-A_d)A_d^{-1} + \tau_{\sigma}A_d^{-1} \frac{\delta A}{\delta t} \right]
$$
in which $h$ is the thickness of the vessel wall, $\mu$ is Poisson's ratio, and $E$ is Young's modulus of the vessel. $\tau_{\epsilon}$ and $\tau_{\sigma}$ represent the relaxation times for viscoelastic vessels at constant strain $\epsilon$ and constant stress $\sigma$, and the subscript $d$ represents quantities at the diastole phase.<sup>1</sup> In our simplified model, we will eventually set $\tau_\epsilon = \tau_\sigma = 0$.

The solutions to the time-domain governing equations are as follows:
$$
\begin{aligned}
Q(x,t) &= Q_d + q(x,t) \\
P(x,t) &= P_d + p(x,t) \\
A(x,t) &= A_d(x) + a(x,t)
\end{aligned}
$$
in which $Q_d$, $P_d$, and $A_d(x)$ are the constant blood flow rate, the constant blood pressure, and the lumen cross-section area at the diastole phase, respectively. Making the assumption that $a \ll A_0$ and noting that $p=q=a=0$ at the diastole phase, we obtain new forms of the governing equations in terms of $q$, $p$, and $a$:
$$
\dot{a} + q' = -\psi \tag{continuity equation}
$$
$$
\begin{aligned}
\dot{q} + 2\gamma Q_d A_0^{-1} q' + (2\gamma \theta Q_d A_0^{-1})q - \nu q'' + \rho^{-1} A_0 p' - \gamma Q_d^2 A_0^{-2} a' \\
= A_0 f_x - \gamma(2A_0^{-1} qq' + A_0^{-1} \theta q^2 - 2A_0^{-2}Q_d qa' - A_0^{-2} q^2 a')
\end{aligned} \tag{momentum balance equation}
$$
$$
p + \tau_{\epsilon} \dot{p} = \kappa(a + \tau_{\sigma} \dot{a}) \quad \text{with} \quad \kappa = \frac{1}{2}EHR^{-1} A_0^{-1} (1-\mu^2)^{-1} \tag{constitutive equation}
$$
In these equations, the dot ($\cdot$) and prime ($'$) represent the partial deriatives w.r.t to time $t$ and the spatial coordinate $x$, respectively. We obtain the final forms of the time-domain governing equations as
$$
\mathbf{M}\mathbf{\ddot{u}} + \mathbf{C}\mathbf{\dot{u}} + \mathbf{G}\mathbf{\dot{u}'} + \mathbf{D}\mathbf{u}'' + \mathbf{A}\mathbf{u}' + \mathbf{K}\mathbf{u} = \mathbf{f}_e + \mathbf{f}_{NL}
$$
with the state vector
$$
\mathbf{u}(x,t) = \mathbf{\begin{pmatrix} p(x,t) \\ q(x,t) \end{pmatrix}}
$$

so that $\mathbf{M,C,G,D,A,K}$ are $2 \times 2$ coefficient matricies determined by the linearized governing equations.

To construct the spectral element, we first transform the time-domain governing equations into the frequency-domain governing equations using discrete Fourier transform theory. We represent the time histories of $\mathbf{u}(x,t)$, $\mathbf{f}_e(x,t)$, and $\mathbf{f}_{NL}(x,t)$ in the spectral forms:
$$
\{ \mathbf{u}(x,t), \mathbf{f}_e(x,t), \mathbf{f}_{NL}(x,t) \} = \frac{1}{N} \sum_{n=0}^{N-1} {\mathbf{\bar{u}}(x;\omega_n), \mathbf{\bar{f}_e}(x;\omega_n), \mathbf{\bar{f}_{NL}}(x;\omega_n)} \, e^{i \omega_n t}
$$
where
$$
\mathbf{\bar{u}} = \begin{Bmatrix} \bar{p} \\ \bar{q} \end{Bmatrix}, 
\mathbf{\bar{f}_e} = \begin{Bmatrix} \bar{f}_{e1} \\ \bar{f}_{e2} \end{Bmatrix},
\mathbf{\bar{f}_{NL}} = \begin{Bmatrix} 0 \\ \bar{f}_{NL} \end{Bmatrix}
$$

**This is computational component #1.** We use `numpy.ftt.ftt`. We obtain the frequency-domain governing equations as
$$
\mathbf{D}\mathbf{\bar{u}}'' + \mathbf{H}\mathbf{\bar{u}}' + \mathbf{L}\mathbf{\bar{u}} = \mathbf{\bar{f}}_e + \mathbf{\bar{f}}_{NL}
$$
in which
$$
\mathbf{H} = i \omega_n \mathbf{G} + \mathbf{A}, \quad \mathbf{L} = o \omega_n \mathbf{C} + \mathbf{K} - \omega_n^2 \mathbf{M}
$$

We also define the spectral nodel degrees of greedom (DOFs) vectors as
$$
\mathbf{d}_p = \begin{Bmatrix} \bar{p}_1 \\ \bar{p}_2 \end{Bmatrix} = \begin{Bmatrix} +\bar{p}(0) \\ +\bar{p}(l) \end{Bmatrix}
$$
$$
\mathbf{d}_q = \begin{Bmatrix} \bar{q}_1 \\ \bar{q}_2 \end{Bmatrix} = \begin{Bmatrix} +\bar{q}(0) \\ -\bar{q}(l) \end{Bmatrix}
$$
with the graphic taken from Lee and Jang<sup>1</sup> showing the sign convention.

From the linear governing equation, we obtain a dispersion relation and solve for wavenumbers that represent the forward and backward traveling waves. We consider the linear homogeneous governing equations by setting $\mathbf{\bar{f}}_e = \mathbf{\bar{f}}_{NL} = 0$:
$$
\mathbf{D}\mathbf{\bar{u}}'' + \mathbf{H}\mathbf{\bar{u}}' + \mathbf{L}\mathbf{\bar{u}} = 0
$$

Assume a general solution fo the form $\mathbf{\bar{u}}(x) = \mathbf{a} e^{-ikx}$ and substitute into the linear homogeneous governing equations to obtain a dispersion relation as
$$
\beta_1 k^2 + \beta_2 k + \beta_3 = 0
$$
(specify $\beta$ here)
Then we calculate two wavenumbers from this dispersion relation as
$$
k_{1,2} = \frac{-\beta_2 \pm \sqrt{\beta_2^2 - 4 \beta_1 \beta_3}}{2\beta_1}
$$

Using the corresponding eigenvectors, we obtain a general solution to the linear homogeneous governing equations:
$$
\mathbf{\bar{u}}(x) = \begin{Bmatrix} \bar{p}(x) \\ \bar{q}(x) \end{Bmatrix}
= \mathbf{a}_1 e^{-ik_1 x} + \mathbf{a}_2 e^{-ik_2 x}
= \begin{bmatrix} e^{-ik_1 x} & e^{-ik_2 x} \\ \phi_1 e^{-ik_1 x} & \phi_2 e^{-ik_2 x} \end{bmatrix} \mathbf{c}
$$
with $c = \begin{Bmatrix} c_1 \\ c_2 \end{Bmatrix}$ being a constant vector. We then apply the spectral nodel DOFs to this solution:
$$
\mathbf{\bar{u}}(x) = \mathbf{N})_p(x;\omega_n) \mathbf{d}_p
$$

with $\mathbf{N}_p(x;\omega_n) = \begin{bmatrix} \mathbf{N}_1(x;\omega_n) \\ \mathbf{N}_2(x;\omega_n) \end{bmatrix}$ being the frequency-dependent dynamic shape function matrix. **This is computational element #2 (including the specific derivation?).**

Applying boundary conditions, we arrive at the spectral element equation:
$$
\mathbf{S}(\omega_n) \mathbf{d}_p = \mathbf{d}_q + \mathbf{\hat{f}}_e + \mathbf{\hat{f}}_{NL}(p,q)
$$
with
$$
\mathbf{S}(\omega_n) = \mathbf{Y}^{-1}(\omega_n) \mathbf{X}(\omega_n) \tag{exact dynamic stiffness equation}
$$
$$
\mathbf{X}(\omega_n) = -\int_{0}^{1} \mathbf{N}_p'^T \mathbf{D} \mathbf{N}_p' dx - \int_{0}^{1} \mathbf{N}_p'^T \mathbf{H} \mathbf{N}_p dx + \int_{0}^{1} \mathbf{N}_p^T \mathbf{L} \mathbf{N}_p dx
$$
$$
\mathbf{Y}(\omega_n) = [ \mathbf{N}_p^T (\mathbf{D} \mathbf{N}_p' + \mathbf{H} \mathbf{N}_p)|_0^l \mathbf{T}^{-1} ]
$$

and
$$
\mathbf{\hat{f}}_e = \mathbf{Y}^{-1} \int_{0}^{1} \mathbf{N}_p^T \mathbf{\bar{f}}_e dx
$$
$$
\mathbf{\hat{f}}_{NL}(p,q) = \mathbf{Y}^{-1} \int_{0}^{1} \mathbf{N}_p^T \mathbf{\bar{f}}_{NL}(p,q) dx
$$

**This is computational element #3 (creating the exact dynamic stiffness matrix).**

For a single, uniform spectral element with no nonlinearities, we transform the inlet flow signal from the time-domain to the frequency-domain using FFT, compute the exact dynamic stiffness matrix for a given frequency, solve for unknown boundary conditions in the frequency-domain, and transform to the time-domain using IFFT. **This is computational element #4, with the possible added task of including nonlinearities and using an AFT method instead.**

Finally, to validate our model, we show two simple test cases. In the first, we compare our single element SEM model to a finely discretized FEM model. In the second, we 

**If you need to install or import any additional python packages,  please provide complete installation instructions in the code block below**


In [2]:
# Provide complete installation or import information for external packages or modules here e.g.

#pip install somepackage
# from somepackage import blah
import numpy as numpy

## Implementation [50 pts]

Use the Markdown and Code blocks below to implement and document your methods including figures.  Only the first markdown block will be a grading cell but please add (not copy) cells in this section to organize your work. 

Please make the description of your problem readable by interlacing clear explanatory text with code (again with proper grammar and spelling). 
All code should be well described and commented.

For at least one routine you code below, you should provide a test block (e.g. using `numpy.testing` routines, or a convergence plot) to validate your code.  

An **important** component of any computational paper is to demonstrate to yourself and others that your code is producing correct results.

**Computational component \#1**

At the inlet boundary x = 0, I define simple test signals $p(0,t), q(0,t), f_{e1}(0,t), f_{e2}(0,t)$, and $f_{NL}(0,t)$. 
These are single-frequency sine and cosine functions so that the FFT and IFFT can be verified easily. 
I apply FFT in time to each signal, then apply IFFT, and compute the maximum reconstruction error.


In [19]:
def make_sem_params(L, rho, A0, Qd, gamma, theta, nu, kappa):
    params = {
        'L': L, # vessel length 
        'rho': rho, #blood density
        'A0': A0, # diastolic area
        'Qd': Qd, #diastolic flow 
        'gamma': gamma, #velocity profile param
        'theta': theta, #nonlinear velocity component
        'nu': nu, # kinematic viscosity 
        'kappa': kappa #area-pressure proportion
    }
    params['c1'] = A0 / rho
    params['u'] = 2. * gamma * Qd / A0
    params['alpha'] = 2. * gamma * theta * Qd / A0 
    return params 

#for linear PDE in (p,q), we don't need to compute the forcing term fnl, and MCGDAK/DHL(w) are all independent of p,q
#thus per frequency we can just build S(w), solve S(w)d_p and get d_q, then IFFT back to time 
#below is the linear SEM method before building the AFT to test the nonlinear SEM 

#SEM physical parameters based on Lee & Jang (elastic, linear)

#geometry
R = 0.0086 # m lumen radius = 0.86 cm
h = 0.000602 # m wall thickness = 0.0602 cm
A0 = numpy.pi * R**2 #m^2 diastolic cross-sectional area

#vessel length (we model a 4 cm segment)
L = 0.04 # m

#fluid properties
rho = 1055.0 #kg/m^3 blood density (1.055 g/cm^3)
nu  = 4.6e-6 #m^2/s kinematic viscosity (0.046 cm^2/s)

#velocity profile parameters
gamma = 1. / 3. # parabolic profile
theta = 0.
#mean inlet flow rate
Qd = 14e-6 #m^3/s (14 cm^3/s)

#wall material properties
E  = 1.4812e6 #Pa, Young's modulus
mu = 0.25 #poisson's ratio

#linearized area–pressure coefficient we derived for simplified model 
kappa = 0.5 * E * h / (R * A0 * (1.0 - mu**2))

#build parameter dictionary
params = make_sem_params(
    L=L,
    rho=rho,
    A0=A0,
    Qd=Qd,
    gamma=gamma,
    theta = theta,
    nu=nu,
    kappa=kappa
)

print(params)



{'L': 0.04, 'rho': 1055.0, 'A0': 0.0002323521926595011, 'Qd': 1.4e-05, 'gamma': 0.3333333333333333, 'theta': 0.0, 'nu': 4.6e-06, 'kappa': 237992732.93008944, 'c1': 2.2023904517488257e-07, 'u': 0.040168905774049656, 'alpha': 0.0}


In [7]:
def sem_fft_ifft():

    #time grid stuff 
    T = 1.0
    N = 1000
    t = numpy.linspace(0.0, T, N, endpoint=False)
    dt = t[1] - t[0]
    
    #test signals for p,q,f_e1,f_e2, f_nl
    p=numpy.sin(2*numpy.pi*t)
    q=numpy.cos(2*numpy.pi*t)
    fe1=numpy.sin(4*numpy.pi*t)
    fe2=numpy.zeros_like(t)
    fnl=numpy.zeros_like(t)
    
    #FFT for p,q,fe1,fe2,fnl
    p_fft=numpy.fft.fft(p)
    q_fft=numpy.fft.fft(q)
    fe1_fft=numpy.fft.fft(fe1)
    fe2_fft=numpy.fft.fft(fe2)
    fnl_fft=numpy.fft.fft(fnl)
    freq=numpy.fft.fftfreq(N, dt)
    omega=2*numpy.pi*freq
    
    #IFFT for same variables as above
    p_t= numpy.fft.ifft(p_fft).real
    q_t= numpy.fft.ifft(q_fft).real
    fe1_t= numpy.fft.ifft(fe1_fft).real
    fe2_t= numpy.fft.ifft(fe2_fft).real
    fnl_t= numpy.fft.ifft(fnl_fft).real
    
    #errors between fft and ifft
    ep = numpy.max(numpy.abs(p - p_t))
    eq = numpy.max(numpy.abs(q - q_t))
    e1 = numpy.max(numpy.abs(fe1 - fe1_t))
    e2 = numpy.max(numpy.abs(fe2 - fe2_t))
    eNL = numpy.max(numpy.abs(fnl - fnl_t))
    
    #print(ep, eq, e1, e2, eNL)
    return t, omega, (p_fft, q_fft, fe1_fft, fe2_fft, fnl_fft)

sem_fft_ifft()
#errors for each are of order 10^-16, so FFT to IFFT is numerically exact to machine precision. 

(array([0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
        0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017,
        0.018, 0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026,
        0.027, 0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035,
        0.036, 0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044,
        0.045, 0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053,
        0.054, 0.055, 0.056, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062,
        0.063, 0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07 , 0.071,
        0.072, 0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08 ,
        0.081, 0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089,
        0.09 , 0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098,
        0.099, 0.1  , 0.101, 0.102, 0.103, 0.104, 0.105, 0.106, 0.107,
        0.108, 0.109, 0.11 , 0.111, 0.112, 0.113, 0.114, 0.115, 0.116,
        0.117, 0.118, 0.119, 0.12 , 0.121, 0.122, 0.123, 0.124, 0.125,
      

## Discussion [15 pts]

Evaluate the results of your project including 
* Why should I believe that your numerical results are correct (convergence, test cases etc)?
* Did the project work (in your opinion)?
* If yes:  what would be the next steps to try
* If no:  Explain why your approach did not work and what you might do to fix it.


YOUR ANSWER HERE